In [3]:
from uuid import uuid3, NAMESPACE_DNS

In [4]:
from hashlib import md5

In [5]:
from src.util import get_obj_ref

In [6]:
import pandas as pd

In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
from src.util import set_context, comp_path

In [9]:
from src.models import run_estimator_cv, PARAMS_SKOPT, get_oof_predictions

/home/lpkirwin/miniconda/envs/kaggle/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [10]:
from skopt import BayesSearchCV
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_predict

In [11]:
set_context("titanic")

Files in data directory:
______

titanic/
    clean.pkl
    raw/
        train.csv
        test.csv
        gender_submission.csv
______



In [12]:
str(uuid3(NAMESPACE_DNS, repr(LGBMClassifier())))[-6:]

'6e0ed3'

In [13]:
md5(repr(LGBMClassifier(n_estimators=3)).encode("utf-8")).hexdigest()[-8:]

'83068d77'

In [14]:
str(type(LGBMClassifier())).split(".")

["<class 'lightgbm", 'sklearn', "LGBMClassifier'>"]

In [15]:
str(LGBMClassifier().__class__)

"<class 'lightgbm.sklearn.LGBMClassifier'>"

In [16]:
get_obj_ref(LGBMClassifier())

'LGBMClassifier_04a6eff6'

In [17]:
df = pd.read_pickle(comp_path("clean.pkl"))
df.shape

(1309, 13)

In [18]:
tr = df[~df._test]
tr.shape

(891, 13)

In [19]:
tr.sample(2)

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,_test
665,32.0,NaN,S,73.500000,"Hickman, Mr. Lewis",0,666,2,male,2,0.0,S.O.C. 14879,False
753,23.0,NaN,S,7.894531,"Jonkoff, Mr. Lalio",0,754,3,male,0,0.0,349204,False


In [20]:
target = "Survived"
X_cols = set(tr.columns) - {target}

In [21]:
X = tr[X_cols]
y = tr[target]

In [22]:
est = LGBMClassifier()

In [23]:
est.fit(X, y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [24]:
PARAMS_SKOPT["lgb_small_trees"]

{'n_estimators': Integer(low=50, high=2000),
 'max_depth': Integer(low=1, high=8),
 'num_leaves': Integer(low=4, high=32),
 'learning_rate': Real(low=0.0001, high=10, prior='log-uniform', transform='identity')}

In [25]:
est.best_score_

defaultdict(dict, {})

In [27]:
class BayesSearchCV2(BayesSearchCV):
    
    def __init__(self, estimator, search_spaces, optimizer_kwargs=None,
                 n_iter=50, scoring=None, fit_params=None, n_jobs=1,
                 n_points=1, iid=True, refit=True, cv=None, verbose=0,
                 pre_dispatch='2*n_jobs', random_state=None,
                 error_score='raise', return_train_score=False):

        self.search_spaces = search_spaces
        self.n_iter = n_iter
        self.n_points = n_points
        self.random_state = random_state
        self.optimizer_kwargs = optimizer_kwargs
        self._check_search_space(self.search_spaces)
        self.fit_params = fit_params

        super(BayesSearchCV, self).__init__(
            estimator=estimator, scoring=scoring,
            n_jobs=n_jobs, iid=iid, refit=refit, cv=cv, verbose=verbose,
            pre_dispatch=pre_dispatch, error_score=error_score,
            return_train_score=return_train_score)

In [28]:
est = BayesSearchCV2(
    LGBMClassifier(),
    PARAMS_SKOPT["lgb_big_trees"],
    scoring="roc_auc",
    cv=5,
    n_iter=25,
    n_jobs=-1,
    verbose=1,
)

In [29]:
est.fit(X, y)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.5s finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.4s finished


BayesSearchCV2(cv=5, error_score='raise',
               estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                        colsample_bytree=1.0,
                                        importance_type='split',
                                        learning_rate=0.1, max_depth=-1,
                                        min_child_samples=20,
                                        min_child_weight=0.001,
                                        min_split_gain=0.0, n_estimators=100,
                                        n_jobs=-1, num_leaves=31,
                                        objective=None, random_state=None,
                                        reg_alpha=0.0, reg_lambda=0.0,
                                        sile...
               fit_params=None, iid=True, n_iter=25, n_jobs=-1, n_points=1,
               optimizer_kwargs=None, pre_dispatch='2*n_jobs',
               random_state=None, refit=True, return_train_score=False,
  

In [30]:
repr(est.best_estimator_)

"LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,\n               importance_type='split', learning_rate=0.004056945310337203,\n               max_depth=100, min_child_samples=20, min_child_weight=0.001,\n               min_split_gain=0.0, n_estimators=500, n_jobs=-1, num_leaves=500,\n               objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,\n               silent=True, subsample=1.0, subsample_for_bin=200000,\n               subsample_freq=0)"

In [31]:
est.best_score_

0.8754841319657813

In [32]:
pd.DataFrame(est.cv_results_)

,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,param_num_leaves,params
0,0.703953,0.832016,0.860829,0.867246,0.904749,0.833531,0.068972,1,0.163291,0.024828,0.025346,0.005761,0.297724,116,129,264,"{'learning_rate': 0.2977244321179457, 'max_dep..."
1,0.828524,0.825823,0.886631,0.883422,0.899757,0.864708,0.031254,1,0.377628,0.079894,0.027012,0.005481,0.002639,121,346,423,"{'learning_rate': 0.002639012482648878, 'max_d..."
2,0.741107,0.824506,0.851471,0.878476,0.917026,0.842299,0.059231,1,0.067018,0.009533,0.020696,0.003708,0.736015,198,35,339,"{'learning_rate': 0.7360145123609051, 'max_dep..."
3,0.722793,0.820949,0.866644,0.864840,0.904074,0.835640,0.062405,1,0.457857,0.066750,0.032746,0.006395,0.157341,168,416,313,"{'learning_rate': 0.1573409267736521, 'max_dep..."
4,0.732016,0.826350,0.853275,0.864305,0.899757,0.834943,0.056719,1,0.408994,0.078167,0.032638,0.006316,0.192573,199,390,221,"{'learning_rate': 0.19257253426871168, 'max_de..."
5,0.833729,0.823518,0.883422,0.876872,0.912169,0.865806,0.032855,1,0.599798,0.115798,0.029647,0.005853,0.012987,188,479,344,"{'learning_rate': 0.012987441506001809, 'max_d..."
6,0.732213,0.814888,0.844719,0.843449,0.891123,0.825088,0.052577,1,0.229351,0.045483,0.027801,0.006957,1.070652,139,407,250,"{'learning_rate': 1.0706519831804433, 'max_dep..."
7,0.822200,0.812385,0.858222,0.851203,0.898071,0.848291,0.030157,1,0.175801,0.031694,0.028024,0.006561,0.000179,197,136,212,"{'learning_rate': 0.0001790838790312364, 'max_..."
8,0.721344,0.819763,0.854545,0.865374,0.908797,0.833738,0.063108,1,0.215923,0.050164,0.036828,0.016467,0.405299,117,156,419,"{'learning_rate': 0.4052991151205078, 'max_dep..."
9,0.616601,0.591765,0.713102,0.660829,0.744131,0.665060,0.057076,1,0.049227,0.012001,0.023606,0.006100,6.277901,120,47,258,"{'learning_rate': 6.27790119756941, 'max_depth..."


In [34]:
cross_val_predict(est.best_estimator_, X, y, cv=5, method="predict_proba")[:, 1]

array([0.10208162, 0.84251394, 0.34053742, 0.84251394, 0.17294969,
       0.13715396, 0.10391543, 0.36130271, 0.36396468, 0.83091414,
       0.62831723, 0.67176353, 0.22430298, 0.08425937, 0.56168607,
       0.67176353, 0.26373112, 0.13224226, 0.51568949, 0.59879854,
       0.05815338, 0.08043304, 0.50800132, 0.22887952, 0.42450412,
       0.11005445, 0.04785602, 0.19128479, 0.59360634, 0.04827733,
       0.24285582, 0.78023817, 0.59879854, 0.1421908 , 0.25596264,
       0.23587212, 0.04785829, 0.19579534, 0.53790005, 0.51313868,
       0.33281318, 0.7354459 , 0.05074379, 0.83091414, 0.49587347,
       0.190341  , 0.13135231, 0.59879854, 0.1273358 , 0.53790005,
       0.26373112, 0.10363613, 0.84251394, 0.7354459 , 0.10391543,
       0.10884462, 0.7354459 , 0.13926464, 0.7354459 , 0.08227747,
       0.11268771, 0.84251394, 0.14740429, 0.26373112, 0.10884462,
       0.13736069, 0.7354459 , 0.22430298, 0.42166984, 0.11560954,
       0.12946108, 0.11686448, 0.11597001, 0.13674021, 0.13402